# Doge

训练 [Wonderful Matrices](https://arxiv.org/abs/2407.16958) 论文中提出的 `Doge` 小型语言模型.
Doge在 Transformers 的框架基础上, 将序列变换部分的 `Multi-Head Attention` 替换为 `Inner Function Attention`, 将状态变换部分的 `MLP` 替换为 `CDMoE` . 

Train the `Doge` small language model proposed in the paper [Wonderful Matrices](https://arxiv.org/abs/2407.16958).
Doge is based on the Transformers framework, replacing the `Multi-Head Attention` in the sequence transformation part with `Inner Function Attention`, and replacing the `MLP` in the state transformation part with `CDMoE`.

![doge_architecture](../assets/doge_architecture.png)

## PreTrain

### 下载预训练与微调数据集
### Download Pre-Training and Fine-Tuning Datasets

预训练数据集, 我们选取了 `fineweb-edu-dedup` 高质量文本, `cosmopedia-v2` 合成指令数据集, 并补充 `python-edu` 与 `open-web-math` 来保证模型的代码与数学能力. 

For the pre-training dataset, we selected the high-quality text `fineweb-edu-dedup`, the synthetic instruction dataset `cosmopedia-v2`, and supplemented it with `python-edu` and `open-web-math` to ensure the model's code and mathematical capabilities.

> 请注意: 由于数据集过大, 至少需要 2TB 的存储空间.

> Note: Due to the large size of the dataset, at least 2TB of storage space is required.

In [ ]:
# 填写保存路径, 缓存路径和进程数
# Padding save path, cache path and number of processes
!python ./examples/pretrain/scripts/download_datasets.py --save_dir ./datasets --cache_dir ./cache --num_proc 1

### 预处理数据集
### Preprocess Datasets

我们需要使用 `tokenizer` 将数据集转为模型可接受的 `input_ids` 与 `attention_mask`.
Doge 使用 `LlamaTokenizer` , 该 tokenizer 词表大小为 `32768` , 使用 `[INST]` 与 `[/INST]` 标记指令. 它还包括工具标记, 但是我们不会在这里使用它们.
像 cosmopedia-v2 这样的数据集就包括 `prompt` 与 `text` 两个字段, 我们就将他们标记为用户内容与助手内容.

We need to use the `tokenizer` to convert the dataset into `input_ids` and `attention_mask` that the model can accept.
Doge uses the `LlamaTokenizer`, which has a vocabulary size of `32768`, and uses the `[INST]` and `[/INST]` tags to mark instructions. It also includes utility tokens, but we won't use them here.
Datasets like cosmopedia-v2 include two fields, `prompt` and `text`, which we will mark as user content and assistant content.

```python
conversation = [
    {"role": "user", "content": prompt},
    {"role": "assistant", "content": text},
]
return tokenizer.apply_chat_template(conversation, tokenize=True, padding='max_length', truncation=True, max_length=MAX_LENGTH, return_dict=True)
```

当然你也可以自行加入一些指令提示.

Of course, you can also add some instruction prompts yourself.

```python
conversation = [
    {"role": "user", "content": "Who are you?"},
    {"role": "assistant", "content": "I am an AI assistant named `Doge`, I am a language model trained by `Shi Jingze` based on the `Doge` architecture, and my task is to provide appropriate answers and support to users based on their questions and requests."},
    {"role": "user", "content": prompt},
    {"role": "assistant", "content": text},
]
```

In [ ]:
# 填写数据集路径, 保存路径, 分词器路径, token数量, 最大长度和进程数
# Padding dataset path, save path, tokenizer path, token number, max length and number of processes
# NOTE: 我们只保留 100B tokens 的数据集, 比例为 fineweb-edu:cosmopedia-v2:python-edu:open-web-math = 7:2:0.5:0.5, 如果你需要训练更大的模型, 请自行增加数据集的规模
# NOTE: We only keep 100B tokens dataset, the ratio of fineweb-edu:cosmopedia-v2:python-edu:open-web-math = 7:2:0.5:0.5, if you need to train larger model, please increase the scale of the dataset by yourself
!python ./examples/pretrain/scripts/preprocess_datasets.py --datasets_dir ./datasets --save_dir ./datasets --tokenizer_path ./examples/tokenizer --tokens 100000000000 --max_length 2048 --num_proc 16

### 合并数据集
### Concatenate Datasets

我们将 fineweb-edu_tokenized, cosmopedia-v2, python-edu 和 open-web-math 数据集合并为 `pretrain` 数据集.
然后将它们打乱顺序 `seed=233` , 并拆分出来 `1,000` 个样本作为测试集.

We combine the fineweb-edu_tokenized, cosmopedia-v2, python-edu, and open-web-math datasets into the `pretrain` dataset.
Then shuffle the order `seed=233`, and split out `1,000` samples as the test set.

In [ ]:
# 填写数据集路径, 保存路径和进程数
# Padding dataset path, save path and number of processes
!python ./examples/pretrain/scripts/concatenate_datasets.py --datasets_dir ./datasets --save_dir ./datasets --num_proc 16

### 配置模型参数
### Configure Model Parameters

我们配置一个 `22M` 的小型模型, 进行训练测试.

| Params | n_layers | d_model | d_ff           | n_heads | n_inner_v | n_exprets | n_expert_heads | n_expert_pre_head |
|--------|----------|---------|----------------|---------|-----------|-----------|----------------|-------------------|
| 22M    | 4        | 256     | 1024           | 2       | 2         | 1024      | 1              | 2                 |
| 77M    | 8        | 512     | 2048           | 4       | 4         | 2048      | 2              | 4                 |
| 197M   | 12       | 768     | 3072           | 6       | 6         | 3072      | 3              | 6                 |
| 400M   | 16       | 1024    | 4096           | 8       | 8         | 4096      | 4              | 8                 |
| 1.2B   | 24       | 1536    | 6144           | 12      | 12        | 6144      | 6              | 12                |
| 2.8B   | 32       | 2048    | 8192           | 16      | 16        | 8192      | 8              | 16                |

- n_layers 是模型的解码器层数
- d_model 是模型的隐藏层维度
- n_heads 是InnerFuncAttn的多头注意力头数 d_model // n_heads 最好保持在 64 以上
- n_inner_v 是InnerFuncAttn的V的数量 d_model // n_inner_v 最好保持在 64 以上

### 配置预训练超参数
### Configure Pre-Training Hyperparameters

| Params | tokens | num_train_epochs | per_epoch_max_steps | accumulate_steps | learning_rate | warmup_ratio | weight_decay | min_lr_rate |
|--------|--------|------------------|---------------------|------------------|---------------|--------------|--------------|-------------|
| 22M    | 4B     | 1                | 20,000              | 128              | 8e-4          | 0.1          | 0.01         | 0.1         |
| 77M    | 20B     | 1               | 40,000              | 256              | 6e-4          | 0.1          | 0.01         | 0.1         |
| 197M   | 80B    | 1                | 80,000              | 512              | 5e-4          | 0.1          | 0.01         | 0.1         |
| 400M   | 320B    | 1               | 160,000             | 1024             | 4e-4          | 0.1          | 0.01         | 0.1         |

### 训练
### Training

In [ ]:
# 你需要指定配置文件路径, 日志路径, 输出路径和分词器路径, 如果需要的话, 还可以指定检查点继续训练 --resume_from_checkpoint
!python ./examples/pretrain/scripts/pretrain.py --config_path ./examples/pretrain/configs/doge_22M.yaml --logging_dir ./logs --output_dir ./results --tokenizer_path ./examples/tokenizer --resume_from_checkpoint <path_to_checkpoint>

## FineTune

### 下载微调数据集
### Download Fine-Tuning Datasets

微调数据集, 我们选取了 `smoltalk` 合成指令数据集, 来进行监督微调.

For the fine-tuning dataset, we selected the synthetic instruction dataset `smoltalk` for supervised fine-tuning.

In [ ]:
# 填写保存路径, 缓存路径和进程数
# Padding save path, cache path and number of processes
!python ./examples/finetune/scripts/download_datasets.py --save_dir ./datasets --cache_dir ./cache --num_proc 1

### 处理微调数据集

### Process Fine-Tuning Datasets

我们将微调数据集应用于`聊天模板`.

We'll apply Fine-Tuning datasets with `chat templete` .



In [ ]:
# 填充数据集存放路径 数据集保存路径、分词器路径、进程数量 
# Padding dataset path, save path, tokenizer path, number process.
!python ./examples/finetune/scripts/preprocess_datasets.py --datasets_dir ./datasets --save_dir ./datasets --tokenizer_path ./examples/tokenizer --num_proc 8

### 微调
### Fine-Tuning

In [ ]:
# 你需要指定预训练模型名称或者路径, 配置文件路径, 日志路径和输出路径, 如果需要的话, 还可以指定检查点继续训练 --resume_from_checkpoint
!python ./examples/finetune/scripts/sft.py --pretrained_model_name_or_path JingzeShi/Doge-22M --config_path ./examples/finetune/configs/doge_22M.yaml --logging_dir ./logs --output_dir ./results --resume_from_checkpoint <path_to_checkpoint>

## 评估
## Evaluation

我们先安装 `miniconda` .

First, install `miniconda`.

```bash
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
bash Miniconda3-latest-Linux-x86_64.sh
```

然后创建评估环境.

Then create an evaluation environment.

```bash
conda create -n lighteval python=3.10.12 
conda activate lighteval
pip install lighteval[accelerate]
```

最后我们运行评估脚本.

Finally, we run the evaluation script.

如果你使用 Linux, 你可以运行以下命令.

if you use Linux, you can run the following command.

```bash
bash ./examples/evaluate/eval_downstream_tasks.sh
```

如果你使用 Windows, 你可以运行以下命令.

if you use Windows, you can run the following command.

```bash
. ./examples/evaluate/eval_downstream_tasks.ps1
```

> 注意: 脚本中的 MODEL 也可以填入保存的检查点路径, 只需要注册保存即可运行.

> NOTE: The MODEL in the script can also be filled with the saved checkpoint path, just need to register the save to run.